###### CFG

In [ ]:
# ====================================================
# CFG
# ====================================================

class CFG:
    num_workers=4
    path="../input/4-1-xlage-fake/"
    config_path=path+'config.pth'
    model="microsoft/deberta-xlarge"
    batch_size=32
    fc_dropout=0.0
    max_len=466
    seed=42
    n_fold=5
    trn_fold=[0,1,2,3,4]

class CFG_2:
    num_workers=4
    path="../input/nbme-deberta-v3-large-fgm-pseudo/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=48
    fc_dropout=0.0
    max_len=370
    seed=42
    n_fold=5
    trn_fold=[0,1,2,3,4]
    

class CFG_3:
    num_workers=4
    path="../input/output-4-5-largev3-new-fgm/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=48
    fc_dropout=0.0
    max_len=354
    seed=42
    n_fold=5
    trn_fold=[0,2,3]
    
class CFG_4:
    num_workers=4
    path="../input/4-28-xlarge-fold-pseudo/"
    config_path=path+'config.pth'
    model="microsoft/deberta-xlarge"
    batch_size=32
    fc_dropout=0.0
    max_len=466
    seed=42
    n_fold=5
    trn_fold=[0,1,2,3,4]

class CFG_5:
    num_workers=4
    path="../input/nmbe-v3large-0427-models/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=48
    fc_dropout=0.0
    max_len=370
    seed=42
    n_fold=5
    trn_fold=[0,1,2,3,4]

In [ ]:
import shutil
from pathlib import Path

transformers_path = Path("/opt/conda/lib/python3.7/site-packages/transformers")

input_dir = Path("../input/debert-largev3-token" + "/deberta-v2-3-fast-tokenizer")

convert_file = input_dir / "convert_slow_tokenizer.py"
conversion_path = transformers_path/convert_file.name

if conversion_path.exists():
    conversion_path.unlink()

shutil.copy(convert_file, transformers_path)
deberta_v2_path = transformers_path / "models" / "deberta_v2"

for filename in ['tokenization_deberta_v2.py', 'tokenization_deberta_v2_fast.py', "deberta__init__.py"]:
    if str(filename).startswith("deberta"):
        filepath = deberta_v2_path/str(filename).replace("deberta", "")
    else:
        filepath = deberta_v2_path/filename
    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir/filename, filepath)

# Library

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

# os.system('pip uninstall -y transformers')
# os.system('python -m pip install --no-index --find-links=../input/nbme-pip-wheels transformers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# ====================================================
# tokenizer
# ====================================================
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

from transformers.models.deberta_v2 import DebertaV2TokenizerFast
tokenizer = DebertaV2TokenizerFast.from_pretrained('../input/large-v3-tokenizer/tokenizer')
    
# tokenizer = AutoTokenizer.from_pretrained(CFG.model)
CFG_2.tokenizer = tokenizer

CFG_3.tokenizer = tokenizer

CFG_4.tokenizer = AutoTokenizer.from_pretrained(CFG_4.path+'tokenizer/')

CFG_5.tokenizer = tokenizer


# Helper functions for scoring

In [ ]:
# From https://www.kaggle.com/theoviel/evaluation-metric-folds-baseline

def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    return f1_score(truths, preds)


def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.

    Args:
        spans (list of lists of two ints): Spans.

    Returns:
        np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
    return binary


def span_micro_f1(preds, truths):
    """
    Micro f1 on spans.

    Args:
        preds (list of lists of two ints): Prediction spans.
        truths (list of lists of two ints): Ground truth spans.

    Returns:
        float: f1 score.
    """
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
    return micro_f1(bin_preds, bin_truths)

In [ ]:
def create_labels_for_scoring(df):
    # example: ['0 1', '3 4'] -> ['0 1; 3 4']
    df['location_for_create_labels'] = [ast.literal_eval(f'[]')] * len(df)
    for i in range(len(df)):
        lst = df.loc[i, 'location']
        if lst:
            new_lst = ';'.join(lst)
            df.loc[i, 'location_for_create_labels'] = ast.literal_eval(f'[["{new_lst}"]]')
    # create labels
    truths = []
    for location_list in df['location_for_create_labels'].values:
        truth = []
        if len(location_list) > 0:
            location = location_list[0]
            for loc in [s.split() for s in location.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                truth.append([start, end])
        truths.append(truth)
    return truths


def get_char_probs(texts, predictions, tokenizer):
    results = [np.zeros(len(t)) for t in texts]
    for i, (text, prediction) in enumerate(zip(texts, predictions)):
        encoded = tokenizer(text, 
                            add_special_tokens=True,
                            return_offsets_mapping=True)
        for idx, (offset_mapping, pred) in enumerate(zip(encoded['offset_mapping'], prediction)):
            start = offset_mapping[0]
            end = offset_mapping[1]
            results[i][start:end] = pred
    return results


def get_results(char_probs, th=0.5, texts=None):
    results = []
    for idx, char_prob in enumerate(char_probs):
        result = np.where(char_prob >= th)[0] + 1
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        
        if len(result) > 0:
            if result[0][0] == 1:
                result[0][0] = 0
                
        te = texts[idx]
        encoded = CFG_2.tokenizer(te,
                            add_special_tokens=True,
                            return_offsets_mapping=True)

       
        result_new = []
        if len(result) > 0:
            for r in result:
                str_tmp = te[min(r):max(r)]

                pos_b = min(r)
                if pos_b > 0:
                    txt_b = te[pos_b]
                    txt_a = te[pos_b - 1]

                    for idx, token in enumerate(encoded.encodings[0].tokens):
                        map_tmp = encoded['offset_mapping'][idx]
                        if  pos_b >= map_tmp[0] and pos_b <= map_tmp[1]:
                            if txt_a in token and txt_b in token:
                                r[0] = r[0] - 1
                                break

                # pos_d = max(r)
                # if pos_d < len(te) - 1:
                #     txt_a = te[pos_d - 1]
                #     txt_b = te[pos_d]
                #
                #     for idx, token in enumerate(encoded.encodings[0].tokens):
                #         map = encoded['offset_mapping'][idx]
                #         if pos_d >= map[0] and pos_d <= map[1]:
                #             if txt_a in token and txt_b in token:
                #                 r[-1] = r[-1] + 1
                #                 break

                result_new.append(r)

        result = result_new

        result = [f"{min(r)} {max(r)}" for r in result]
        result = ";".join(result)
        results.append(result)
    return results



def get_predictions(results):
    predictions = []
    for result in results:
        prediction = []
        if result != "":
            for loc in [s.split() for s in result.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                prediction.append([start, end])
        predictions.append(prediction)
    return predictions

In [ ]:
def get_results_pp2(char_probs, th=0.5, texts=None):
    results = []
    for idx, char_prob in enumerate(char_probs):
        text = texts[idx]                            # 对照文本
        result = np.where(char_prob >= th)[0] + 1
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        temp = []
        for r in result:
            start = min(r)
            end = max(r)
            if start <= 1:           # 修复丢失文本第 0 个字符的情况
                start = 0
            elif start == end:
                start -= 1
            elif re.match(r'^[a-zA-Z0-9]$', text[start - 1]) and (text[start - 2] in ['\t', '\n', '\r', ',', '.', ':', ';', '-', '+', '"', '(', '/', '&', '*']):   # 修复没有空格而丢失单词第一个字符的情况
                start -= 1
            else:
                pass
            temp.append(f"{start} {end}")    
        temp = ";".join(temp)    
        results.append(temp)
    return results

# Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = span_micro_f1(y_true, y_pred)
    return score


def get_logger(filename='inference'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# OOF

# Data Loading

In [ ]:
# ====================================================
# Data Loading
# ====================================================
test = pd.read_csv('../input/nbme-score-clinical-patient-notes/test.csv')
submission = pd.read_csv('../input/nbme-score-clinical-patient-notes/sample_submission.csv')
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features
features = preprocess_features(features)
patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')

print(f"test.shape: {test.shape}")
display(test.head())
print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())

In [ ]:
test = test.merge(features, on=['feature_num', 'case_num'], how='left')
test = test.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(test.head())

# Dataset

In [ ]:
# ====================================================
# Dataset
# ====================================================
def prepare_input_fast(cfg, text, feature_text, batch_max_len):
    inputs = cfg.tokenizer(text, feature_text, 
                           add_special_tokens=True,
                           max_length=batch_max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDatasetFast(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values
        self.batch_max_len = df['batch_max_length'].values

    def __len__(self):
        return len(self.feature_texts)

    def __getitem__(self, item):
        inputs = prepare_input_fast(self.cfg, 
                               self.pn_historys[item], 
                               self.feature_texts[item],
                               self.batch_max_len[item],
                              )
        return inputs

# Model

In [ ]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, 1)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        return last_hidden_states

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

# inference

In [ ]:
# ====================================================
# inference
# ====================================================
def inference_fn_fast(test_loader, model, device, max_len_in):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
    # for inputs in test_loader:
        bs = len(inputs['input_ids'])
        pred_w_pad = np.zeros((bs, max_len_in, 1))
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        y_preds = y_preds.sigmoid().to('cpu').numpy()
        pred_w_pad[:, :y_preds.shape[1]] = y_preds
        preds.append(pred_w_pad)
    predictions = np.concatenate(preds)
    return predictions


In [ ]:
###### Reduce Padding Inference ######

# sort by token num
input_lengths = []
tk0 = tqdm(zip(test['pn_history'].fillna("").values, test['feature_text'].fillna("").values), total=len(test))
for text, feature_text in tk0:
    length = len(CFG.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
    input_lengths.append(length)
test['input_lengths'] = input_lengths
length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# sort dataframe
sort_df = test.iloc[length_sorted_idx]

# calc max_len per batch
sorted_input_length = sort_df['input_lengths'].values
batch_max_length = np.zeros_like(sorted_input_length)
bs = CFG.batch_size
for i in range((len(sorted_input_length)//bs)+1):
    batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
sort_df['batch_max_length'] = batch_max_length

In [ ]:
test_dataset = TestDatasetFast(CFG, sort_df)
test_loader = DataLoader(test_dataset,
                      batch_size=CFG.batch_size,
                      shuffle=False,
                      num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

print(len(test_dataset))
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn_fast(test_loader, model, device, CFG.max_len)
    prediction = prediction.reshape((len(test), CFG.max_len))
    ## data re-sort ## 
    prediction = prediction[np.argsort(length_sorted_idx)]
    char_probs = get_char_probs(test['pn_history'].values, prediction, CFG.tokenizer)
    predictions.append(char_probs)
    del model, state, prediction, char_probs; gc.collect()
    torch.cuda.empty_cache()
predictions_xlarge = np.mean(predictions, axis=0)

In [ ]:
test_2 = pd.read_csv('../input/nbme-score-clinical-patient-notes/test.csv')
submission = pd.read_csv('../input/nbme-score-clinical-patient-notes/sample_submission.csv')


features_2 = pd.read_csv('../input/nbme-feature-data/features_with_meanings.csv')
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features
features_2 = preprocess_features(features_2)
features_2['feature_text'] = features_2.apply(lambda row: row['feature_text'] + '[SEP]meaning: ' + row['meaning'], axis=1)
features_2.drop(['meaning'], axis=1, inplace=True)


patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')

print(f"test.shape: {test_2.shape}")
display(test.head())
print(f"features.shape: {features_2.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())

In [ ]:
# ====================================================
# Data Loading
# ====================================================
test_3 = pd.read_csv('../input/nbme-score-clinical-patient-notes/test.csv')
test_4 = pd.read_csv('../input/nbme-score-clinical-patient-notes/test.csv')
test_5 = pd.read_csv('../input/nbme-score-clinical-patient-notes/test.csv')
features_3 = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features
features_3 = preprocess_features(features_3)

print(f"test.shape: {test_3.shape}")
display(test_3.head())
print(f"features.shape: {features_3.shape}")
display(features_3.head())

In [ ]:
test_3 = test_3.merge(features_3, on=['feature_num', 'case_num'], how='left')
test_3 = test_3.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(test_3.head())

In [ ]:
test_4 = test_4.merge(features_3, on=['feature_num', 'case_num'], how='left')
test_4 = test_4.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(test_4.head())

In [ ]:
test_2 = test_2.merge(features_2, on=['feature_num', 'case_num'], how='left')
test_2 = test_2.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(test.head())

In [ ]:
test_5 = test_5.merge(features_2, on=['feature_num', 'case_num'], how='left')
test_5 = test_5.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(test.head())

In [ ]:
###### Reduce Padding Inference ######

# sort by token num
input_lengths = []
tk0 = tqdm(zip(test_2['pn_history'].fillna("").values, test_2['feature_text'].fillna("").values), total=len(test_2))
for text, feature_text in tk0:
    length = len(CFG_2.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
    input_lengths.append(length)
test_2['input_lengths'] = input_lengths
length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# sort dataframe
sort_df = test_2.iloc[length_sorted_idx]

# calc max_len per batch
sorted_input_length = sort_df['input_lengths'].values
batch_max_length = np.zeros_like(sorted_input_length)
bs = CFG_2.batch_size
for i in range((len(sorted_input_length)//bs)+1):
    batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
sort_df['batch_max_length'] = batch_max_length

In [ ]:
test_dataset_2 = TestDatasetFast(CFG_2, sort_df)
test_loader_2 = DataLoader(test_dataset_2,
                      batch_size=CFG_2.batch_size,
                      shuffle=False,
                      num_workers=CFG_2.num_workers, pin_memory=True, drop_last=False)

print(len(test_dataset_2))
predictions_v3_all = []
for fold in CFG_2.trn_fold:
    model_2 = CustomModel(CFG_2, config_path=CFG_2.config_path, pretrained=False)
    
    state_2 = torch.load(CFG_2.path+f"{CFG_2.model.replace('/', '-')}_fold{fold}_best.pth",
                   map_location=torch.device('cpu'))
    
    model_2.load_state_dict(state_2['model'])
    prediction = inference_fn_fast(test_loader_2, model_2, device, CFG_2.max_len)
    prediction = prediction.reshape((len(test_2), CFG_2.max_len))
    ## data re-sort ## 
    prediction = prediction[np.argsort(length_sorted_idx)]
    
    char_probs = get_char_probs(test_2['pn_history'].values, prediction, CFG_2.tokenizer)
    predictions_v3_all.append(char_probs)
    del model_2, state_2, prediction, char_probs; gc.collect()
    torch.cuda.empty_cache()
predictions_v3 = np.mean(predictions_v3_all, axis=0)

In [ ]:
###### Reduce Padding Inference ######

# sort by token num
input_lengths = []
tk0 = tqdm(zip(test_3['pn_history'].fillna("").values, test_3['feature_text'].fillna("").values), total=len(test_3))
for text, feature_text in tk0:
    length = len(CFG_3.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
    input_lengths.append(length)
test_3['input_lengths'] = input_lengths
length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# sort dataframe
sort_df = test_3.iloc[length_sorted_idx]

# calc max_len per batch
sorted_input_length = sort_df['input_lengths'].values
batch_max_length = np.zeros_like(sorted_input_length)
bs = CFG_3.batch_size
for i in range((len(sorted_input_length)//bs)+1):
    batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
sort_df['batch_max_length'] = batch_max_length

In [ ]:
test_dataset_3 = TestDatasetFast(CFG_3, sort_df)
test_loader_3 = DataLoader(test_dataset_3,
                      batch_size=CFG_3.batch_size,
                      shuffle=False,
                      num_workers=CFG_3.num_workers, pin_memory=True, drop_last=False)

print(len(test_dataset_3))
predictions_largev3_all_3 = []
for fold in CFG_3.trn_fold:
    model_3 = CustomModel(CFG_3, config_path=CFG_3.config_path, pretrained=False)
    
    state_3 = torch.load(CFG_3.path+f"{CFG_3.model.replace('/', '-')}_fold{fold}_best.pth",
               map_location=torch.device('cpu'))
    
    model_3.load_state_dict(state_3['model'])
    
    prediction = inference_fn_fast(test_loader_3, model_3, device, CFG_3.max_len)
    prediction = prediction.reshape((len(test_3), CFG_3.max_len))
    ## data re-sort ## 
    prediction = prediction[np.argsort(length_sorted_idx)]
    
    char_probs = get_char_probs(test_3['pn_history'].values, prediction, CFG_3.tokenizer)
    predictions_largev3_all_3.append(char_probs)
    del model_3, state_3, prediction, char_probs; gc.collect()
    torch.cuda.empty_cache()

predictions_v3_2 = np.mean(predictions_largev3_all_3, axis=0)

In [ ]:
###### Reduce Padding Inference ######

# sort by token num
input_lengths = []
tk0 = tqdm(zip(test_4['pn_history'].fillna("").values, test_4['feature_text'].fillna("").values), total=len(test_4))
for text, feature_text in tk0:
    length = len(CFG_4.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
    input_lengths.append(length)
test_4['input_lengths'] = input_lengths
length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# sort dataframe
sort_df = test_4.iloc[length_sorted_idx]

# calc max_len per batch
sorted_input_length = sort_df['input_lengths'].values
batch_max_length = np.zeros_like(sorted_input_length)
bs = CFG_4.batch_size
for i in range((len(sorted_input_length)//bs)+1):
    batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
sort_df['batch_max_length'] = batch_max_length

In [ ]:
test_dataset_4 = TestDatasetFast(CFG_4, sort_df)
test_loader_4 = DataLoader(test_dataset_4,
                      batch_size=CFG_4.batch_size,
                      shuffle=False,
                      num_workers=CFG_4.num_workers, pin_memory=True, drop_last=False)

print(len(test_dataset_4))
predictions_xlarge_all_new = []
for fold in CFG_4.trn_fold:
    model_4 = CustomModel(CFG_4, config_path=CFG_4.config_path, pretrained=False)
    
    state_4 = torch.load(CFG_4.path+f"{CFG_4.model.replace('/', '-')}_fold{fold}_best.pth",
               map_location=torch.device('cpu'))
    
    model_4.load_state_dict(state_4['model'])
    
    prediction = inference_fn_fast(test_loader_4, model_4, device, CFG_4.max_len)
    prediction = prediction.reshape((len(test_4), CFG_4.max_len))
    ## data re-sort ## 
    prediction = prediction[np.argsort(length_sorted_idx)]
    
    char_probs = get_char_probs(test_4['pn_history'].values, prediction, CFG_4.tokenizer)
    predictions_xlarge_all_new.append(char_probs)
    del model_4, state_4, prediction, char_probs; gc.collect()
    torch.cuda.empty_cache()

predictions_xlarge_new = np.mean(predictions_xlarge_all_new, axis=0)

In [ ]:
###### Reduce Padding Inference ######

# sort by token num
input_lengths = []
tk0 = tqdm(zip(test_5['pn_history'].fillna("").values, test_5['feature_text'].fillna("").values), total=len(test_5))
for text, feature_text in tk0:
    length = len(CFG_5.tokenizer(text, feature_text, add_special_tokens=True)['input_ids'])
    input_lengths.append(length)
test_5['input_lengths'] = input_lengths
length_sorted_idx = np.argsort([-len_ for len_ in input_lengths])

# sort dataframe
sort_df = test_5.iloc[length_sorted_idx]

# calc max_len per batch
sorted_input_length = sort_df['input_lengths'].values
batch_max_length = np.zeros_like(sorted_input_length)
bs = CFG_5.batch_size
for i in range((len(sorted_input_length)//bs)+1):
    batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
sort_df['batch_max_length'] = batch_max_length

In [ ]:
test_dataset_5 = TestDatasetFast(CFG_5, sort_df)
test_loader_5 = DataLoader(test_dataset_5,
                      batch_size=CFG_5.batch_size,
                      shuffle=False,
                      num_workers=CFG_5.num_workers, pin_memory=True, drop_last=False)

print(len(test_dataset_5))
predictions_largev3_h_new = []
for fold in CFG_5.trn_fold:
    model_5 = CustomModel(CFG_5, config_path=CFG_5.config_path, pretrained=False)
    
    state_5 = torch.load(CFG_5.path+f"{CFG_5.model.replace('/', '-')}_fold{fold}_best.pth",
               map_location=torch.device('cpu'))
    
    model_5.load_state_dict(state_5['model'])
    
    prediction = inference_fn_fast(test_loader_5, model_5, device, CFG_5.max_len)
    prediction = prediction.reshape((len(test_5), CFG_5.max_len))
    ## data re-sort ## 
    prediction = prediction[np.argsort(length_sorted_idx)]
    
    char_probs = get_char_probs(test_5['pn_history'].values, prediction, CFG_5.tokenizer)
    predictions_largev3_h_new.append(char_probs)
    del model_5, state_5, prediction, char_probs; gc.collect()
    torch.cuda.empty_cache()

predictions_largev3_h = np.mean(predictions_largev3_h_new, axis=0)

In [ ]:
predictions_1 = predictions_xlarge * 0.4 + predictions_v3 * 0.32 + predictions_v3_2 * 0.28
predictions_2 = predictions_xlarge_new * 0.75 + predictions_largev3_h * 0.25

predictions = predictions_1 * 0.45 + predictions_2 * 0.55

results = get_results_pp2(predictions, 0.44, test['pn_history'].values)
submission['location'] = results
display(submission.head())
submission[['id', 'location']].to_csv('submission.csv', index=False)